In [1]:
# Import Counter object
from collections import Counter
# Import pandas for reading data
import pandas as pd
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Handling data imbalancing
from imblearn.over_sampling import SMOTE # doctest: +NORMALIZE_WHITESPACE
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
# Import train_test_split function
from sklearn.model_selection import train_test_split
# Import K-neareast 
from sklearn.neighbors import KNeighborsClassifier
# Import SGD Classifier
from sklearn.linear_model import SGDClassifier
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics


C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 

In [2]:
# Dealing with segmented data
#"""select count(1) from "AI_LAGORTA"."CAMP_AUG15_Y_CBM1""""

from sqlalchemy import create_engine
import cx_Oracle


host='192.168.113.80'
port=1521
sid='bbidb'
user='system'
password='oraclebbi'
sid = cx_Oracle.makedsn(host, port, sid=sid)

def oracle_connect(host=host, sid=sid, user=user, password=password):
    cstr = 'oracle://{user}:{password}@{sid}/'.format (
        user = user ,
        password = password ,
        sid = sid
    )

    engine = create_engine (
        cstr ,
        convert_unicode = False ,
        pool_recycle = 10 ,
        pool_size = 50 ,
        echo =False
    )

    return engine.connect()

In [3]:
# retrieve database table
con = oracle_connect()
df = pd.read_sql_query('SELECT * FROM AI_LAGORTA.CAMP_AUG15_Y_CBM2', con = con)

In [4]:
# handling ordinal data
v_customer_value_enc = OrdinalEncoder ( categories = [ [ 'LVC' , 'HVC' ] ] )
nationality_group_enc = OrdinalEncoder ( categories = [ [ 'Low' , 'Med' , "High" ] ] )
df [ 'v_customer_value$' ] = v_customer_value_enc.fit_transform (df [ 'v_customer_value$' ].values.reshape ( -1 , 1 ) )
df [ 'nationality_group' ] = nationality_group_enc.fit_transform (df [ 'nationality_group' ].values.reshape ( -1 , 1 ) )
# separate predictors and target
X = df.drop(['pk_msisdn_id', 'target_all'], axis=1)
y = df['target_all'].values

In [5]:
# split dataset into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y) # 70% training and 30% test


In [6]:
len(y_train), len(y_test)

(16672, 7146)

In [7]:
# Training class count
import numpy as np

unique, counts = np.unique(y_train, return_counts=True)
percent = (counts / len(y_train)) * 100
print(np.asarray((unique, percent)).T)

[['0' 96.42514395393475]
 ['1' 3.574856046065259]]


In [8]:
# Testing class count
unique, counts = np.unique(y_test, return_counts=True)
percent = counts 
print(np.asarray((unique, percent)).T)

[['0' 6891]
 ['1' 255]]


In [9]:
# Handling data imbalancing
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Original dataset shape %s' % Counter(y_train))
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({'0': 16076, '1': 596})
Resampled dataset shape Counter({'0': 16076, '1': 16076})


In [10]:
# Random Forest Model
model = SGDClassifier(loss='modified_huber', shuffle=True)
model.fit(X_res, y_res)



SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [11]:
# Training Data Metrics
target_names = ['class 0', 'class 1']
print(metrics.classification_report(y_res, model.predict(X_res), target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.58      0.48      0.53     16076
     class 1       0.56      0.66      0.60     16076

    accuracy                           0.57     32152
   macro avg       0.57      0.57      0.56     32152
weighted avg       0.57      0.57      0.56     32152



In [12]:
# Testing Data Metrics
target_names = ['class 0', 'class 1']
print(metrics.classification_report(y_test, model.predict(X_test), target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.98      0.47      0.64      6891
     class 1       0.05      0.71      0.09       255

    accuracy                           0.48      7146
   macro avg       0.51      0.59      0.36      7146
weighted avg       0.94      0.48      0.62      7146

